In [1]:
import numpy as np

index = np.load("/home/jovyan/Sample_Based_Extension/UNSW/best_defense_indices.npy", allow_pickle=True)
index

array([['baseline', 0, 'Def1-11 Combined', 0.9923313825446824,
        0.9911800925969834, array([9, 3, 9, ..., 9, 9, 9])],
       ['BIM', 0.01, 'Def1-11 Combined', 0.9891551359556379,
        0.987484040917074, array([9, 3, 9, ..., 9, 9, 9])],
       ['BIM', 0.1, 'Def1-11 Combined', 0.9746251239350361,
        0.9705688736459472, array([1, 6, 1, ..., 1, 1, 1])],
       ['BIM', 0.2, 'Def1-11 Combined', 0.9594545980995165,
        0.9538946564599001, array([6, 6, 6, ..., 6, 6, 6])],
       ['BIM', 0.3, 'Def1-11 Combined', 0.919233840187416,
        0.9113183206844415, array([6, 6, 6, ..., 6, 6, 6])],
       ['FGSM', 0.01, 'Def1-11 Combined', 0.9889357819094331,
        0.9872271983425641, array([9, 3, 9, ..., 9, 9, 9])],
       ['FGSM', 0.1, 'Def1-11 Combined', 0.9680708250344385,
        0.9628361221532343, array([1, 6, 1, ..., 1, 1, 1])],
       ['FGSM', 0.2, 'Def1-11 Combined', 0.9461968395469023,
        0.9384213027985543, array([1, 6, 1, ..., 1, 1, 1])],
       ['FGSM', 0.3, 'Def1

In [2]:
attack_models = [
    ("BIM", [0.1]),
    ("FGSM", [0.1]),
    ("PGD", [0.1]),
    ("DF", [0.1]),
    ("AutoPGD", [0.1]),
    ("ZOO", [0.1]),
    ("CaFA", [0.1]),
    ("SINIFGSM", [0.1]),
    ("VNIFGSM", [0.1]),
]

all_adv_examples = []

for attack_name, epsilons in attack_models:
    for epsilon in epsilons:
        filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
        x_test_adv = np.load(filename)
        all_adv_examples.append(x_test_adv)

x_test_adv_0_1 = np.concatenate(all_adv_examples, axis=0)


In [3]:
y_labels = []

for row in index:
    attack_name, eps = row[0], row[1]
    if eps == 0.1:
        y_array = row[-1]
        y_labels.append(y_array)

y_label_0_1 = np.concatenate(y_labels, axis=0)

In [4]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(y_label_0_1)

y_label_0_1 = label_encoder.transform(y_label_0_1)

for index, class_label in enumerate(label_encoder.classes_):
    print(f"defense '{class_label}' encode {index}")

defense '1' encode 0
defense '2' encode 1
defense '3' encode 2
defense '4' encode 3
defense '5' encode 4
defense '6' encode 5
defense '7' encode 6
defense '8' encode 7
defense '9' encode 8
defense '10' encode 9
defense '11' encode 10


In [5]:
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import entropy
import numpy as np

def stratified_random_sample(X, y, fraction=None, count=None, random_state=42):
    assert (fraction is not None) ^ (count is not None), "Specify either fraction or count"

    rng = np.random.RandomState(seed=random_state)
    X_out, y_out = [], []

    for label in np.unique(y):
        X_c = X[y == label]
        y_c = y[y == label]

        n_total = len(X_c)
        n_sample = int(n_total * fraction) if fraction is not None else min(count, n_total)
        if n_sample == 0:
            n_sample = 1

        idx = rng.choice(n_total, size=n_sample, replace=False)

        X_out.append(X_c[idx])
        y_out.append(y_c[idx])

    return np.concatenate(X_out, axis=0), np.concatenate(y_out, axis=0)


def classwise_uncertainty_sampling_by_fraction(X_pool, y_pool, model, fraction=0.2, random_state=42):
    X_selected = []
    y_selected = []
    selected_indices = []

    classes = np.unique(y_pool)
    probs = model.predict_proba(X_pool)
    rng = np.random.RandomState(seed=random_state)

    for c in classes:
        idx_c = np.where(y_pool == c)[0]
        X_c = X_pool[idx_c]
        probs_c = probs[idx_c]


        n_c = len(idx_c)
        n_select = max(1, int(n_c * fraction))

        if n_c == 0:
            continue

        uncertainties = entropy(probs_c.T)  # shape: (n_samples_in_class,)
        sorted_idx = np.argsort(uncertainties)[-n_select:]  # top-k uncertain

        top_k_idx = idx_c[sorted_idx]

        selected_indices.extend(top_k_idx)
        X_selected.append(X_pool[top_k_idx])
        y_selected.append(y_pool[top_k_idx])

    selected_indices = np.array(selected_indices)
    X_selected = np.concatenate(X_selected, axis=0)
    y_selected = np.concatenate(y_selected, axis=0)

    remaining_idx = np.setdiff1d(np.arange(len(X_pool)), selected_indices)
    X_rest = X_pool[remaining_idx]
    y_rest = y_pool[remaining_idx]

    return X_rest, X_selected, y_rest, y_selected

X_pool = x_test_adv_0_1 
y_pool = y_label_0_1
X_train_init, y_train_init = stratified_random_sample(X_pool, y_pool, fraction=0.1, random_state=42)

clf = RandomForestClassifier(random_state=42)
clf.fit(X_train_init, y_train_init)


_, X_50, _, y_50 = classwise_uncertainty_sampling_by_fraction(
    x_test_adv_0_1, y_label_0_1, clf, fraction=0.5
)



In [6]:
X_50.shape

(512867, 56)

In [7]:
from xgboost import XGBClassifier

print('xgb')
xgb50 = XGBClassifier()
xgb50.fit(X_50, y_50)

print("RF")
from sklearn.ensemble import RandomForestClassifier
rf50 = RandomForestClassifier()
rf50.fit(X_50, y_50)

print("DT")
from sklearn.tree import DecisionTreeClassifier
dt50 = DecisionTreeClassifier()
dt50.fit(X_50, y_50)

xgb
RF
DT


DecisionTreeClassifier()

In [8]:
import time
import numpy as np

y_test = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')

attack_models = [
    ("baseline", [0]),
    ("BIM", [0.01, 0.2, 0.3]),
    ("FGSM", [0.01, 0.2, 0.3]),
    ("PGD", [0.01, 0.2, 0.3]),
    ("DF", [0.01, 0.2, 0.3]),
    ("AutoPGD", [0.01, 0.2, 0.3]),
    ("ZOO", [0.01, 0.2, 0.3]),
    ("CaFA", [0.01, 0.2, 0.3]),
    ("SINIFGSM", [0.01, 0.2, 0.3]),
    ("VNIFGSM", [0.01, 0.2, 0.3]),
]

base_path_template = "/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack}_eps_{eps}.npy"

from sklearn.model_selection import train_test_split

for attack_name, epsilons in attack_models:
    start_time = time.time()
    all_adv_examples = []
    all_y = []
    
    if attack_name != "baseline":
        for epsilon in epsilons:
            filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
            x_test_adv = np.load(filename)
            all_adv_examples.append(x_test_adv)
            all_y.append(y_test)
    
        X = np.concatenate(all_adv_examples, axis=0)
        y = np.concatenate(all_y, axis=0)
    else:
        X = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_test.npy')
        y = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')

    y_pred = xgb50.predict(X)
    print(y_pred.shape)
    unique_classes, count = np.unique(y_pred, return_counts=True)
    print(unique_classes, count)
    for class_num in unique_classes:
        indices = np.where(y_pred == class_num)[0]
        X_class = X[indices]
        y_class = y[indices]
        print(X_class.shape, y_class.shape)
    
        x_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/XGB_ActiveLearning/UNSW_Input50/x_test_adv_{attack_name}_Def{class_num+1}.npy"
        y_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/XGB_ActiveLearning/UNSW_Input50/y_test_adv_{attack_name}_Def{class_num+1}.npy"
        
        np.save(x_filename, X_class)
        np.save(y_filename, y_class)
    
        print(f"Save {class_num} to {attack_name}_Def{class_num+1}.npy")

    end_time = time.time()
    result = end_time - start_time
    print(f"Execution Time: {result:.6f} seconds")
    

(113971,)
[ 0  1  2  3  4  5  6  7  8  9 10] [70686    96  1851   442    62  2783     1   190 37627   155    78]
(70686, 56) (70686,)
Save 0 to baseline_Def1.npy
(96, 56) (96,)
Save 1 to baseline_Def2.npy
(1851, 56) (1851,)
Save 2 to baseline_Def3.npy
(442, 56) (442,)
Save 3 to baseline_Def4.npy
(62, 56) (62,)
Save 4 to baseline_Def5.npy
(2783, 56) (2783,)
Save 5 to baseline_Def6.npy
(1, 56) (1,)
Save 6 to baseline_Def7.npy
(190, 56) (190,)
Save 7 to baseline_Def8.npy
(37627, 56) (37627,)
Save 8 to baseline_Def9.npy
(155, 56) (155,)
Save 9 to baseline_Def10.npy
(78, 56) (78,)
Save 10 to baseline_Def11.npy
Execution Time: 0.205993 seconds
(341913,)
[ 0  1  2  3  4  5  6  7  8  9 10] [158339     89   1305   2885    502 134736      1    382  43624     24
     26]
(158339, 56) (158339,)
Save 0 to BIM_Def1.npy
(89, 56) (89,)
Save 1 to BIM_Def2.npy
(1305, 56) (1305,)
Save 2 to BIM_Def3.npy
(2885, 56) (2885,)
Save 3 to BIM_Def4.npy
(502, 56) (502,)
Save 4 to BIM_Def5.npy
(134736, 56) (134736,

In [9]:
from sklearn.model_selection import train_test_split

for attack_name, epsilons in attack_models:
    start_time = time.time()
    all_adv_examples = []
    all_y = []
    
    if attack_name != "baseline":
        for epsilon in epsilons:
            filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
            x_test_adv = np.load(filename)
            all_adv_examples.append(x_test_adv)
            all_y.append(y_test)
    
        X = np.concatenate(all_adv_examples, axis=0)
        y = np.concatenate(all_y, axis=0)
    else:
        X = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_test.npy')
        y = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')

    y_pred = rf50.predict(X)
    print(y_pred.shape)
    unique_classes, count = np.unique(y_pred, return_counts=True)
    print(unique_classes, count)
    for class_num in unique_classes:
        indices = np.where(y_pred == class_num)[0]
        X_class = X[indices]
        y_class = y[indices]
        print(X_class.shape, y_class.shape)
    
        x_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF_ActiveLearning/UNSW_Input50/x_test_adv_{attack_name}_Def{class_num+1}.npy"
        y_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF_ActiveLearning/UNSW_Input50/y_test_adv_{attack_name}_Def{class_num+1}.npy"
        
        np.save(x_filename, X_class)
        np.save(y_filename, y_class)
    
        print(f"Save {class_num} to {attack_name}_Def{class_num+1}.npy")

    end_time = time.time()
    result = end_time - start_time
    print(f"Execution Time: {result:.6f} seconds")
    

(113971,)
[ 0  1  2  3  4  5  7  8  9 10] [58972   141  1620   752   114  3371   327 48448   142    84]
(58972, 56) (58972,)
Save 0 to baseline_Def1.npy
(141, 56) (141,)
Save 1 to baseline_Def2.npy
(1620, 56) (1620,)
Save 2 to baseline_Def3.npy
(752, 56) (752,)
Save 3 to baseline_Def4.npy
(114, 56) (114,)
Save 4 to baseline_Def5.npy
(3371, 56) (3371,)
Save 5 to baseline_Def6.npy
(327, 56) (327,)
Save 7 to baseline_Def8.npy
(48448, 56) (48448,)
Save 8 to baseline_Def9.npy
(142, 56) (142,)
Save 9 to baseline_Def10.npy
(84, 56) (84,)
Save 10 to baseline_Def11.npy
Execution Time: 2.534119 seconds
(341913,)
[0 2 3 4 5 8 9] [126048    677    848      6 180241  34081     12]
(126048, 56) (126048,)
Save 0 to BIM_Def1.npy
(677, 56) (677,)
Save 2 to BIM_Def3.npy
(848, 56) (848,)
Save 3 to BIM_Def4.npy
(6, 56) (6,)
Save 4 to BIM_Def5.npy
(180241, 56) (180241,)
Save 5 to BIM_Def6.npy
(34081, 56) (34081,)
Save 8 to BIM_Def9.npy
(12, 56) (12,)
Save 9 to BIM_Def10.npy
Execution Time: 3.357560 seconds

In [11]:
from sklearn.model_selection import train_test_split

for attack_name, epsilons in attack_models:
    start_time = time.time()
    all_adv_examples = []
    all_y = []
    
    if attack_name != "baseline":
        for epsilon in epsilons:
            filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
            x_test_adv = np.load(filename)
            all_adv_examples.append(x_test_adv)
            all_y.append(y_test)
    
        X = np.concatenate(all_adv_examples, axis=0)
        y = np.concatenate(all_y, axis=0)
    else:
        X = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_test.npy')
        y = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')

    y_pred = dt50.predict(X)
    print(y_pred.shape)
    unique_classes, count = np.unique(y_pred, return_counts=True)
    print(unique_classes, count)
    for class_num in unique_classes:
        indices = np.where(y_pred == class_num)[0]
        X_class = X[indices]
        y_class = y[indices]
        print(X_class.shape, y_class.shape)
    
        x_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/DT_ActiveLearning/UNSW_Input50/x_test_adv_{attack_name}_Def{class_num+1}.npy"
        y_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/DT_ActiveLearning/UNSW_Input50/y_test_adv_{attack_name}_Def{class_num+1}.npy"
        
        np.save(x_filename, X_class)
        np.save(y_filename, y_class)
    
        print(f"Save {class_num} to {attack_name}_Def{class_num+1}.npy")

    end_time = time.time()
    result = end_time - start_time
    print(f"Execution Time: {result:.6f} seconds")
    

(113971,)
[ 0  1  2  3  4  5  7  8  9 10] [68050   298  1548  2148   166  6664   350 34574    70   103]
(68050, 56) (68050,)
Save 0 to baseline_Def1.npy
(298, 56) (298,)
Save 1 to baseline_Def2.npy
(1548, 56) (1548,)
Save 2 to baseline_Def3.npy
(2148, 56) (2148,)
Save 3 to baseline_Def4.npy
(166, 56) (166,)
Save 4 to baseline_Def5.npy
(6664, 56) (6664,)
Save 5 to baseline_Def6.npy
(350, 56) (350,)
Save 7 to baseline_Def8.npy
(34574, 56) (34574,)
Save 8 to baseline_Def9.npy
(70, 56) (70,)
Save 9 to baseline_Def10.npy
(103, 56) (103,)
Save 10 to baseline_Def11.npy
Execution Time: 0.136096 seconds
(341913,)
[ 0  1  2  3  4  5  6  7  8  9 10] [170917    415   4709   6346    847  83830   3160  36220  34317    748
    404]
(170917, 56) (170917,)
Save 0 to BIM_Def1.npy
(415, 56) (415,)
Save 1 to BIM_Def2.npy
(4709, 56) (4709,)
Save 2 to BIM_Def3.npy
(6346, 56) (6346,)
Save 3 to BIM_Def4.npy
(847, 56) (847,)
Save 4 to BIM_Def5.npy
(83830, 56) (83830,)
Save 5 to BIM_Def6.npy
(3160, 56) (3160,)


In [12]:
import numpy as np


attack_names = [
    "baseline", 
    "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

all_attack_list = []

model_name_list = ["XGB", "RF", "DT"]

for m_name in model_name_list:
    for attack_name in attack_names:
        each = []
        num = 0
        each.append(f"{m_name}50")
        for i in range(1, 12):
            y_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/{m_name}_ActiveLearning/UNSW_Input50/x_test_adv_{attack_name}_Def{i}.npy"
            try:
                y = np.load(y_filename)
                num+=y.shape[0]
                each.append(y.shape[0])
            except FileNotFoundError:
                each.append(0)
        each.append(num)
        all_attack_list.append(each)


In [13]:
import pandas as pd
all_attack_list = pd.DataFrame(all_attack_list)
all_attack_list.to_csv("./Dynamite_Num_Attack_Input50_ActiveLearning.csv")

In [14]:
all_attack_list

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,XGB50,70686,96,1851,442,62,2783,1,190,37627,155,78,113971
1,XGB50,158339,89,1305,2885,502,134736,1,382,43624,24,26,341913
2,XGB50,114360,78,1309,2860,58,179475,4,441,43295,23,10,341913
3,XGB50,158339,89,1305,2885,502,134736,1,382,43624,24,26,341913
4,XGB50,75022,528,771,299,43,245182,579,17408,1961,41,79,341913
5,XGB50,178187,269,3997,1649,585,136293,1,738,19705,258,231,341913
6,XGB50,201489,299,5618,1425,186,7824,4,565,123838,431,234,341913
7,XGB50,290980,78,392,936,93,18725,1,138,29251,978,341,341913
8,XGB50,144407,80,445,2066,217,142125,1,47,52477,27,21,341913
9,XGB50,146391,80,1389,2283,97,143368,1,149,48105,29,21,341913
